# FastAPI Testing Notebook

이 노트북은 구축한 FastAPI 애플리케이션을 테스트하고 분석하기 위한 도구입니다.

## 기능:
- API 엔드포인트 테스트
- 데이터베이스 연결 확인
- JWT 토큰 생성 및 검증
- 성능 측정

In [ ]:
# 필요한 라이브러리 import
import requests
import json
import time
from datetime import datetime, timedelta

# FastAPI 서버 URL
BASE_URL = "http://localhost:8000"

print("FastAPI Testing Environment Setup Complete")
print(f"Base URL: {BASE_URL}")

In [ ]:
# 헬스 체크 테스트
def test_health_check():
    try:
        response = requests.get(f"{BASE_URL}/health")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        return response.status_code == 200
    except Exception as e:
        print(f"Health check failed: {e}")
        return False

# 헬스 체크 실행
print("Testing Health Check Endpoint...")
health_ok = test_health_check()
print(f"Health Check: {'✓ PASSED' if health_ok else '✗ FAILED'}")

In [ ]:
# 상세 헬스 체크 테스트
def test_detailed_health():
    try:
        response = requests.get(f"{BASE_URL}/api/v1/health/detailed")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2)}")
        return response.status_code == 200
    except Exception as e:
        print(f"Detailed health check failed: {e}")
        return False

print("Testing Detailed Health Check...")
detailed_health_ok = test_detailed_health()
print(f"Detailed Health Check: {'✓ PASSED' if detailed_health_ok else '✗ FAILED'}")

In [ ]:
# 사용자 생성 테스트
def test_create_user():
    user_data = {
        "email": "test@example.com",
        "username": "testuser",
        "password": "securepassword123",
        "is_active": True,
        "is_superuser": False
    }
    
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/users/",
            json=user_data,
            headers={"Content-Type": "application/json"}
        )
        
        print(f"Status Code: {response.status_code}")
        if response.status_code == 201:
            print(f"User created: {json.dumps(response.json(), indent=2)}")
            return True, response.json()
        else:
            print(f"Error: {response.text}")
            return False, None
    except Exception as e:
        print(f"User creation failed: {e}")
        return False, None

print("Testing User Creation...")
user_created, user_data = test_create_user()
print(f"User Creation: {'✓ PASSED' if user_created else '✗ FAILED'}")

In [ ]:
# 로그인 테스트
def test_login():
    login_data = {
        "username": "testuser",
        "password": "securepassword123"
    }
    
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/auth/token",
            data=login_data,  # OAuth2PasswordRequestForm expects form data
            headers={"Content-Type": "application/x-www-form-urlencoded"}
        )
        
        print(f"Status Code: {response.status_code}")
        if response.status_code == 200:
            token_data = response.json()
            print(f"Login successful: {json.dumps(token_data, indent=2)}")
            return True, token_data.get('access_token')
        else:
            print(f"Login failed: {response.text}")
            return False, None
    except Exception as e:
        print(f"Login test failed: {e}")
        return False, None

print("Testing User Login...")
login_success, access_token = test_login()
print(f"User Login: {'✓ PASSED' if login_success else '✗ FAILED'}")

if access_token:
    print(f"Access Token (first 20 chars): {access_token[:20]}...")

In [ ]:
# 인증된 사용자 정보 가져오기
def test_get_user_info(token):
    if not token:
        print("No token available")
        return False
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(f"{BASE_URL}/api/v1/users/me", headers=headers)
        
        print(f"Status Code: {response.status_code}")
        if response.status_code == 200:
            user_info = response.json()
            print(f"User Info: {json.dumps(user_info, indent=2)}")
            return True
        else:
            print(f"Error: {response.text}")
            return False
    except Exception as e:
        print(f"Get user info failed: {e}")
        return False

print("Testing Get User Info (Authenticated)...")
user_info_success = test_get_user_info(access_token)
print(f"Get User Info: {'✓ PASSED' if user_info_success else '✗ FAILED'}")

In [ ]:
# API 성능 테스트
def performance_test(endpoint, num_requests=10):
    print(f"\nPerformance Testing: {endpoint}")
    print("-" * 40)
    
    times = []
    success_count = 0
    
    for i in range(num_requests):
        start_time = time.time()
        try:
            response = requests.get(f"{BASE_URL}{endpoint}")
            end_time = time.time()
            
            if response.status_code == 200:
                success_count += 1
                times.append(end_time - start_time)
        except Exception as e:
            print(f"Request {i+1} failed: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        print(f"Requests: {num_requests}")
        print(f"Successful: {success_count}")
        print(f"Success Rate: {success_count/num_requests*100:.1f}%")
        print(f"Average Response Time: {avg_time*1000:.2f} ms")
        print(f"Min Response Time: {min_time*1000:.2f} ms")
        print(f"Max Response Time: {max_time*1000:.2f} ms")
    else:
        print("No successful requests")

# 성능 테스트 실행
performance_test("/health")
performance_test("/api/v1/health/")